# Test name: qa_test_01

##### Reason for test:  
Try a very simple example with two supplies, two demands for each, everything in one axis to give an obvious visual split between the supply areas for the two demands. 

##### Parameters

Two supplies - vertically aligned - 300km apart

Two demands per supply - vertically aligned - 50km directly above and below supplies

Capacities - equal supply to demand points


##### Predicted outcome:

The two pairs of demand for each supply will be associated together and there will be a clear divide halfway between the two supplies. 



In [33]:
import pandas as pd
import psycopg2
import random
import os
from utilities import generate_point_dict 
from utilities import locate_supplies
from utilities import calculate_distances
dir = os.getcwd()
parent_directory = os.path.split(os.path.split(dir)[0])[0]

An example of what the csv data should look like (see column naming)

In [34]:
pd.read_csv(parent_directory + "/web/data/datasets/data.csv", nrows=2)

,demand,demand_id,demand_lat,demand_lng,demand_name,supply,supply_id,supply_lat,supply_lng,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,11.662019,0,50.920128,-2.670739,Green Ln,35.689500,0,51.749314,-0.240863,Roehyde Way,158.883333,192.761855,235.458
1,11.662019,0,50.920128,-2.670739,Green Ln,28.249443,1,50.815128,-2.273901,Deer Park,48.433333,30.269113,50.560


The method for locating demands will change depending on the scenario and the locate_demands function may be different for each different test.

In [35]:
def locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn):
    demands = []
    supply_counter = 0

    for supply_dict in supply_df.to_dict(orient="records"):
        this_supply = {"lat":supply_dict["supply_lat"], "lng": supply_dict["supply_lng"]}
        for i in range(num_demands_per_supply):
            distance_km = random.choice(dem_distances_km)
            bearing_degrees = dem_bearings_degrees[i]
            this_demand = generate_point_dict(this_supply, bearing_degrees, distance_km, conn)
            this_demand["demand"] = random.choice(demand_vol)
            this_demand["demand_name"] = "demand" + str(supply_counter + i)
            demands.append(this_demand)
        supply_counter = supply_counter + num_demands_per_supply

    demands_df = pd.DataFrame(demands).reset_index()
    demands_df.columns=["demand_id", "demand", "demand_name", "demand_lat", "demand_lng"]
    return(demands_df)

### Run from here to create a new test data set based on the parameters which can be changed below.

In [36]:
# Naming parameters
output_name = "qa_data_01"

# Supply parameters
num_supplies = 2
sup_distances_km = [0, 300]
sup_bearings_degrees = [0, 0]
supply_vol = [400, 400]

# Demand parameters
num_demands_per_supply = 2
dem_distances_km = [50]
dem_bearings_degrees = [0, 180] # Demands will be taken at these exact degrees for each supply
demand_vol = [200]

# Misc
scale = 1 # Used to scale demand to be a proportion of supply (1 by default)

In [37]:
conn = "host='localhost' dbname='postgres' user='postgres' password=''"
conn = psycopg2.connect(conn)

supply_df = locate_supplies(num_supplies, sup_distances_km, sup_bearings_degrees, supply_vol, conn)
demands_df = locate_demands(supply_df, num_demands_per_supply, dem_distances_km, dem_bearings_degrees, demand_vol, conn)


In [38]:
supply_df

,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,51.501109,-1.242375,400,supply0
1,1,54.196930,-1.242375,400,supply1


In [39]:
demands_df

,demand_id,demand,demand_name,demand_lat,demand_lng
0,0,200,demand0,51.950498,-1.242375
1,1,200,demand1,51.051685,-1.242375
2,2,200,demand2,54.646114,-1.242375
3,3,200,demand3,53.747712,-1.242375


In [8]:
# Scale supply capacity so that supply>demand by "scale" (ONLY RUN IF SCALING REQUIRED)
#supply_df["supply"] = supply_df["supply"]* demands_df["demand"].sum()/supply_df["supply"].sum()*scale

In [40]:
# Cartestian product
supply_df["cart"] = 1
demands_df["cart"] = 1
all_combos = demands_df.merge(supply_df)
all_combos = all_combos.drop("cart", axis=1)
all_combos

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name
0,0,200,demand0,51.950498,-1.242375,0,51.501109,-1.242375,400,supply0
1,0,200,demand0,51.950498,-1.242375,1,54.196930,-1.242375,400,supply1
2,1,200,demand1,51.051685,-1.242375,0,51.501109,-1.242375,400,supply0
3,1,200,demand1,51.051685,-1.242375,1,54.196930,-1.242375,400,supply1
4,2,200,demand2,54.646114,-1.242375,0,51.501109,-1.242375,400,supply0
5,2,200,demand2,54.646114,-1.242375,1,54.196930,-1.242375,400,supply1
6,3,200,demand3,53.747712,-1.242375,0,51.501109,-1.242375,400,supply0
7,3,200,demand3,53.747712,-1.242375,1,54.196930,-1.242375,400,supply1


In [41]:
# Calculate the crowflies distances for each combination
all_combos = calculate_distances(all_combos, conn)
all_combos.head(10)

,demand_id,demand,demand_name,demand_lat,demand_lng,supply_id,supply_lat,supply_lng,supply,supply_name,duration_min,distance_crowflies_km,distance_route_km
0,0,200,demand0,51.950498,-1.242375,0,51.501109,-1.242375,400,supply0,49.983150,49.983150,49.983150
1,0,200,demand0,51.950498,-1.242375,1,54.196930,-1.242375,400,supply1,249.915294,249.915294,249.915294
2,1,200,demand1,51.051685,-1.242375,0,51.501109,-1.242375,400,supply0,49.983180,49.983180,49.983180
3,1,200,demand1,51.051685,-1.242375,1,54.196930,-1.242375,400,supply1,349.881622,349.881622,349.881622
4,2,200,demand2,54.646114,-1.242375,0,51.501109,-1.242375,400,supply0,349.881409,349.881409,349.881409
5,2,200,demand2,54.646114,-1.242375,1,54.196930,-1.242375,400,supply1,49.982967,49.982967,49.982967
6,3,200,demand3,53.747712,-1.242375,0,51.501109,-1.242375,400,supply0,249.915447,249.915447,249.915447
7,3,200,demand3,53.747712,-1.242375,1,54.196930,-1.242375,400,supply1,49.982998,49.982998,49.982998


In [42]:
# Save file to CSV
path = dir + "/qa_data_files/" + str(output_name) + ".csv"
all_combos.to_csv(path, index=False)